In [3]:
pip install pystac-client

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Import PySTAC Client
# Note: For more information see https://pystac-client.readthedocs.io/en/stable/index.html
from pystac_client import Client

In [5]:
# Connect to the catalog
# Note: Check STAC Index (https://stacindex.org) for a list of available catalogs
catalog = Client.open('https://earth-search.aws.element84.com/v1/')
catalog

<Client id=earth-search-aws>

In [6]:
# Get collections
# Note: It will return a generator.
collections = catalog.get_collections()
collections

<generator object Client.get_collections at 0x77d2c81faf80>

In [7]:
# Get the first collection
collection = next(collections)
collection

<CollectionClient id=sentinel-2-pre-c1-l2a>

In [8]:
# Display collection ids
# Note: Because we already got the first item, it won't be listed. You should get the collection again for a full list.
for collection in collections:
    print(collection.id)

cop-dem-glo-30
naip
cop-dem-glo-90
landsat-c2-l2
sentinel-2-l2a
sentinel-2-l1c
sentinel-2-c1-l2a
sentinel-1-grd


In [9]:
# Get collection with a specific id
collection = catalog.get_collection('sentinel-2-l2a')
collection

<CollectionClient id=sentinel-2-l2a>

In [10]:
# Get items in the collection
# Note: It will return a generator.
items = collection.get_items()
items

<generator object CollectionClient.get_items at 0x77d2c80909e0>

In [11]:
# Get the first item
item = next(items)
item

<Item id=S2B_29TLM_20241125_0_L2A>

In [12]:
# Get item with a specific id
item = collection.get_item('S2B_3UXV_20241124_0_L2A')
item

<Item id=S2B_3UXV_20241124_0_L2A>

In [ ]:
# It is better to use search functionality to get specific items
# Note: Empty search will return all items in all collections.
# Note: Search is lazy, i.e. does not retrieve metadata unless required.
results = catalog.search()

In [ ]:
# Display number of items matching the search query
# Note: There are ore then 105 million items in the catalog!
results.matched()

In [ ]:
# Get the first item
item = next(results.items())
item

In [ ]:
# Add collection criterion to reduce the number of matches
results = catalog.search(
    collections=['sentinel-2-c1-l2a']
)

In [ ]:
results.matched()

In [ ]:
# Define an area of interest (AOI) for spatial filtering
# Note: It is a GeoJSON dictionary for a point. It is possible to use others as well, e.g. polygon
# Note: You can use https://geojson.io to draw features and get corresponding GeoJSON.
aoi = {
    "coordinates": [
      6.854093706579846,
      52.237984736137776
    ],
    "type": "Point"
}

In [ ]:
# Refine AOI
# Note: 5 digits is enough for ~1 m accuracy.
aoi = {
    "coordinates": [6.85409, 52.23798],
    "type": "Point"
}

In [ ]:
# Add spatial criterion to reduce the number of matches
results = catalog.search(
    collections=['sentinel-2-c1-l2a'],
    intersects=aoi,
)

In [ ]:
results.matched()

In [ ]:
# Add date range criterion to reduce the number of matches
# Note: You can also specify a date, a partial date (e.g. 2024), or range of partial dates.
results = catalog.search(
    collections=['sentinel-2-c1-l2a'],
    intersects=aoi,
    datetime=['2024-06-01', '2024-06-15'],  # ['2024']  ['2024-06']
)

In [ ]:
results.matched()

In [ ]:
# Get items
# Note: Does not return a generator, so all metadata needs to be retrieved. For many items this make take time.
items = results.item_collection()
items

In [ ]:
# Sort items by date
# Note: For more information see https://pystac-client.readthedocs.io/en/stable/usage.html#sort-extension
results = catalog.search(
    collections=['sentinel-2-c1-l2a'],
    intersects=aoi,
    datetime=['2024-06-01', '2024-06-15'],
    sortby='properties.datetime',
)

In [ ]:
items = results.item_collection()
items

In [ ]:
# Get the first item (now the oldest in the specified date range as it is sorted in ascending order by datetime)
item = items[0]
item

In [ ]:
# Display item datetime
item.datetime

In [ ]:
# Display item bounding box
item.bbox

In [ ]:
# Display item geometry
item.geometry

In [ ]:
# Import GeoPandas to quickly display items on a map
import geopandas

In [ ]:
# Create geodataframe from items
gdf = geopandas.GeoDataFrame.from_features(items)

In [ ]:
gdf

In [ ]:
# Display map of items
# Note: No background layer is visible, because no CRS information was provided.
gdf.explore()

In [ ]:
# Recreate geodataframe with CRS information
gdf = geopandas.GeoDataFrame.from_features(items, crs="epsg:4326")

In [ ]:
# Display map of items
gdf.explore()

In [ ]:
# Display only the first item on the map
gdf[0:1].explore()

In [ ]:
# List assets of the item
item.assets

In [ ]:
# Get URL address of the thumbnail image
# Note: By clicking the link you can see the image in the web browser
# Note: It is a very cloudy image, so let's try to find another one with less clouds
url = item.assets['thumbnail'].href
url

In [ ]:
# Add cloud cover less than 5% criterio
# Note: For more information see https://pystac-client.readthedocs.io/en/stable/usage.html#sort-extension
results = catalog.search(
    collections=['sentinel-2-c1-l2a'],
    intersects=aoi,
    datetime=['2024-06-01', '2024-06-30'],
    query=['eo:cloud_cover<5'],
    sortby='properties.datetime',
)

In [ ]:
results.matched()

In [ ]:
# Get items
items = results.item_collection()
items

In [ ]:
# Map items
gdf = geopandas.GeoDataFrame.from_features(items, crs="epsg:4326")

In [ ]:
gdf.explore()

In [ ]:
# Get the first item
item = items[0]
item

In [ ]:
# Get URL address of the thumbnail image
url = item.assets['thumbnail'].href
url

In [ ]:
# Display thumbnail image inside the notebook
from IPython.display import Image

In [ ]:
Image(url)

In [ ]:
# Download the thumbnail image as 'thumbnail.jpg'
import urllib.request
urllib.request.urlretrieve(url, "thumbnail.jpg")

In [ ]:
# You can display the local copy as well
Image('thumbnail.jpg')

In [ ]:
# Get URL address of the near-infrared asset
# Note: You can download the image locally by clicking the link
url = item.assets['nir'].href
url

In [ ]:
# Download near-infrared image as 'B08.tif'
urllib.request.urlretrieve(url, "B08.tif")

In [ ]:
# Open the image by using rasterio
# Note: It will only read metadata to open the image, not the data.
import rasterio 

src = rasterio.open(url, 'r')
src

In [ ]:
# Display image dimensions
src.shape

In [ ]:
# Display image bounds
src.bounds

In [ ]:
# Display image transform
src.transform

In [ ]:
# Display image CRS
# Note: It is UTM Zone 32N; hence, it is different from the geometry of the item, which is geographic.
src.crs

In [ ]:
# Display image indexes
src.indexes

In [ ]:
# Read the data
# Note: This will read all data; hence, it needs to download all of it, which takes time.
# Note: We can prevent this by reading only the data we need.
data = src.read(1)

In [ ]:
# Import RasterIO Window to be able to specify the area we are interested in for reading
from rasterio.windows import Window

In [ ]:
# Set window as column 0, row 0, width 500, height 500.
window = Window(0, 0, 500, 500)

In [ ]:
# Read data using the window
# Note: It will only read the block(s) overlaping with the window; hence, it will be fast - especially if the window is small
data = src.read(1, window=window)

In [ ]:
# Display data
# Note: It is 16 bit unsinged integer, which needs to be scaled to get the actual reflectance
# Question: Why they didn't provide reflectance values directly?
data

In [ ]:
# Get asset information
asset = item.assets['nir']
asset

In [ ]:
# Get band information from the asset
band = asset.ext.raster.bands[0]
band

In [ ]:
# Scale data according to the band scale and offset information
scaled_data = data * band.scale + band.offset
scaled_data

In [ ]:
# Plot data
# Note: It is nice to read a small area, but it is an arbitrary area. We need to find a way to read a specific area.
import rasterio.plot

In [ ]:
rasterio.plot.show(scaled_data)

In [ ]:
# Import Shapely to enable designation of a custom area by providing corresponding window
# Note: We need some preprocessing, as the CRS of the image is UTM.
import shapely

In [ ]:
# Define area of interest polygon as GeoJSON string
aoi = '''
{
    "coordinates": [
      [
        [
          6.813111434007084,
          52.259751496921496
        ],
        [
          6.813111434007084,
          52.213380204320174
        ],
        [
          6.904167461964676,
          52.213380204320174
        ],
        [
          6.904167461964676,
          52.259751496921496
        ],
        [
          6.813111434007084,
          52.259751496921496
        ]
      ]
    ],
    "type": "Polygon"
}
'''

In [ ]:
# Create feature
feature = shapely.from_geojson(aoi)
feature

In [ ]:
# Get feature bounds
feature.bounds

In [ ]:
# Import PyPROJ to enable projection of geographic coordinates to UTM
import pyproj

In [ ]:
# Convert bounds to UTM
bounds = pyproj.Proj(src.crs).transform_bounds(*feature.bounds)
bounds

In [ ]:
# Create window from projected bounds by using the transformation of the image
import rasterio.windows

In [ ]:
window = rasterio.windows.from_bounds(*bounds, src.transform)
window

In [ ]:
# Read the area of interest
# Note: No scaling for demonstration purposes, you can do it by yourself
data = src.read(1, window=window)
data

In [ ]:
# Display data
rasterio.plot.show(data)

In [ ]:
# Store data
# Note: You should set crs and transform correctly. Using src.transform directly won't work, because it is for the whole image, not the specific area
with rasterio.open(
    'B08_ITC.tif',
    'w',
    driver='GTiff',
    height=data.shape[0],
    width=data.shape[1],
    count=1,
    dtype=data.dtype,
    #crs=src.crs,
    #transform=rasterio.windows.transform(window, src.transform),
) as dst:
    dst.write(data, 1)

In [ ]:
# Using RasterIO is the lowest level data access you can have. 
# But there are better options available.
# See below for the use of stackstac (https://stackstac.readthedocs.io/en/latest/), which combines STAC with XArray
# Try to document each code cell.
# You can install stackstac by entering 'pip install stackstac' in a terminal.

In [ ]:
import stackstac

In [ ]:
stack = stackstac.stack(items, assets=['red', 'green', 'blue'], epsg=32632)

In [ ]:
stack

In [ ]:
lon, lat = 6.85409, 52.23798

In [ ]:
x, y = pyproj.Proj(stack.crs)(lon, lat)

In [ ]:
x, y

In [ ]:
buffer = 2000

In [ ]:
area = stack.loc[..., y+buffer:y-buffer, x-buffer:x+buffer]
area

In [ ]:
data = area.compute()

In [ ]:
data.plot.imshow(row='time', rgb='band', robust=True, size=6)

In [ ]:
import rioxarray

median = area.median(dim="time", keep_attrs=True)
median.rio.to_raster("B08_Area_Median.tif", driver="COG")